In [ ]:
from collections import deque
import math
from datetime import datetime, timedelta
import pandas as pd

In [ ]:
class Visitor: # generate unique Visitor
    def __init__(self, id, itinerary, fast_pass: int, arrival_time, current_action):
        """
        :param id: a unique ID
        :param itinerary: a list of itinerary (where they will go, in sequence)
        :param fast_pass: 0 for regular ticket, 1 for fast-pass ticket
        """
        # Track: current location, current time

        self.id = id
        self.fast_pass = fast_pass 
        self.itinerary = itinerary 

        self.current_time = arrival_time # First set to when they are 'spawned'
        self.current_location = 'Entrance' # spawn at entrance
        self.next_location = None

        self.status = None # "moving", "queuing", "being served", "none"
        self.count_down = None # Count down to next location update, use when travelling, ridig, queueing

    def __repr__(self): # representation method 
        return f"Visitor {self.id, self.fast_pass}"

    def advance_time(self):
        """
        Advances the current time by a specified number of minutes.
        """
        self.current_time += timedelta(minutes=1)

    def find_next_location(self):
        if self.itinerary:
            choice_index = self.itinerary.pop(0)
            choices = attraction_generator_df[attraction_generator_df['index'] == choice_index]['name']
            if not choices.tolist():
                print(choice_index) # debugging
            self.next_location = random.choices(choices.tolist(),k=1)[0] # because random.choices() return a list, therefore need index

    def find_count_down(self):
        matching_rows = paths_df[(paths_df['source'] == self.current_location) & (paths_df['target'] == self.next_location)]
        if not matching_rows.empty:
            self.count_down = matching_rows['distance'].iloc[0]


In [ ]:
class Attraction: # Including shows, rides
    def __init__(self, name: str, ride_duration: int, ride_capacity: int):
        """
        Initialize an attraction with normal and express queues.
        :param name: Name of the attraction.
        :param ride_duration: Duration of the ride
        :param ride_capacity: Number of visitors that can be processed at once.
        """
        self.name = name

        self.holding_visitors = deque() # To store visitors before adding them to regular/fast-pass queue
        self.regular_queue = deque()  
        self.fast_pass_queue = deque()

        self.ride_duration = int(ride_duration)  # Time of each ride
        self.ride_capacity = ride_capacity # Max number of ppl a ride can take

        self.current_time = None  # Simulation time in seconds

        self.riding_visitors = [] # list of visitors being served in the current ride

        self.total_served = 0  # cumulative number of served visitors 
        self.rides_served = 0 # Cumulative number of rides served in the current simulation, Flag for early termination
    
        self.in_service = False # True if ride is happening
        self.last_ride_time = datetime.strptime("10:00", "%H:%M")


    def __repr__(self): # representation method 
        return f"{self.name}"
 
    def advance_time(self):
        """
        Advances the current time by a specified number of minutes.
        """
        self.current_time += timedelta(minutes=1)

    def add_visitor(self, visitor):
        """
        :param visitor: a Visitor object
        """
        visitor.status = "queuing"
        if visitor.fast_pass == 1:
            self.fast_pass_queue.append(visitor)
            print(f"{visitor} added to the fast-pass queue for {self} at time {self.current_time}.") # for debugtest
        else:
            self.regular_queue.append(visitor)
            print(f"{visitor} added to the regular queue for {self} at time {self.current_time}.") # for debugtest

    def process_queue(self):
        if not self.in_service:
                    
            # take min(1/2*ride_capacity visitors out of fast_pass_queue, len(fast_pass_queue))
            if self.fast_pass_queue: # while fast_pass_queue has visitors
                for _ in range(int(min(self.ride_capacity//2, len(self.fast_pass_queue)))):
                    person = self.fast_pass_queue.popleft()
                    if person not in self.riding_visitors:
                        self.riding_visitors.append(person) # Get visitors in fast_pass_queue
                        person.status = "being served"
                        print(f"{person} from Fast Pass Queue for {self} is being served.") # for debugging
                        self.total_served += 1
                        print(f"{self} 's total serving so far: {self.total_served}")  # for debugging
            
            fast_pass_filled_ride = len(self.riding_visitors) # number of fast_pass visitors going to the current ride
            
            # the rest from regular_queue
            if self.regular_queue:
                for _ in range(int(min(self.ride_capacity - fast_pass_filled_ride, len(self.regular_queue)))):
                    person = self.regular_queue.popleft()
                    if person not in self.riding_visitors:
                        self.riding_visitors.append(person) # Get visitors in regular_queue
                        person.status = "being served"
                        print(f"{person} from Regular Queue for {self} is being served.") # for debugging
                        self.total_served += 1
                        print(f"{self} 's total serving so far: {self.total_served}")  # for debugging
            
            if not self.fast_pass_queue and not self.regular_queue:
                print(f"No one is waiting in the queues for {self} at time", self.current_time)

        
        
    def proceed_ride(self): 
        if self.current_time == self.last_ride_time + self.ride_duration: # time to start a ride
            self.rides_served += 1
            self.last_ride_time = self.current_time  
            self.process_queue()
            for visitor in self.riding_visitors:
                visitor.status = "none" # finish the ride
            self.riding_visitors = [] # reset
            
            

    def get_data(self): 
        if math.isnan(self.ride_duration):
            self.ride_duration = 0
        crowd_level = {'fast_pass_queue':len(self.fast_pass_queue), 
                       'regular_queue':len(self.regular_queue)}
        curr_wait_time = {'fast_pass_queue':math.floor(crowd_level['fast_pass_queue']/self.ride_capacity)*self.ride_duration, 
                          'regular_queue':math.floor(crowd_level['regular_queue']/self.ride_capacity)*self.ride_duration}
        return self.current_time, crowd_level, curr_wait_time
    
    